# Playing with Pandas in the Museum: Collection Data Analysis

Pandas is a Python based data analysis library providing similiar functionality to R. This tutorial provides a very speedy introduction based around museum collection data (from the V&A, Tate & MoMa) 

## Pandas: A quick introduction

To quote from the [website](http://pandas.pydata.org/):

   pandas is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.


## Pandas - The Bear Essentials

Data is stored in Pandas as an array. This can either be a 'Series' (one index column, one data column) 
or a DataFrame (multiple columns, possibly with multiple index columns). That's it!

Most of the operations built into Pandas work on translating/tranforming/mapping data in one
or more of these columns in some way, creating another column in the existing DataFrame as an output or generating 
a new Series/DataFrame. By running a series of operations on your input data, you can clean up
your data (probably the majority of your time is spent on this process), run operations on it, export it, visualise it, or archive it in the (compressed, binary) HDF format for speed of retrieval for future operations on the data (Under development is a new more interoperable archive format of '[Feathers](https://blog.rstudio.org/2016/03/29/feather/)' which will be sharable between Pandas and [R](https://www.r-project.org)

## Installing

Comprehensive installation details are available [here](http://pandas.pydata.org/getpandas.html). For Python users, installation in a virtualenv using pip is recommended, assuming supporting libraries are installed.



## Running Pandas

Once installed, pandas is imported in a python session or script as follows:

In [2]:
import numpy
import pandas as pd

## Reading in data

So now we have a data analysis library ready for use, we just need some collections data to analyse. Let's retrieve some object data from the [V&A's Collection](http://collections.vam.ac.uk) using the public [API](http://www.vam.ac.uk/api/):

curl -s http://www.vam.ac.uk/api/json/museumobject/search?q=pandas | jq -r '["id", "object", "title", "artist", "year_start", "place"], (.records[].fields | [.object_number,.object,.title,.artist,.year_start,.place]) | @csv' > pandas.csv


The above Unix shell command runs a API query for 'panda' related objects in the collections, which returns 10 matching records in JSON format (in a custom data structure) . This data is then piped into jq to convert it into a standard CSV format for reading into Pandas (see Matthew Lincoln's excellent introduction to [reshaping JSON & using JQ](http://programminghistorian.github.io/ph-submissions/lessons/json-and-jq.html) for a guide to transforming JSON into CSV and much more!) The jq command pulls out the fields 'object_number', 'object', 'title', 'artist', 'year_start' and 'place' from each object. We also instruct jq to add the folowing column headers to the CSV file which will aid Pandas in labeling the data when it read it in

```
ObjectId,Object,Title,Artist,ProductionDateEarliest,Place
...
```

Converting the data like this to CSV means we can now make use of Pandas built in ability to read CSV files, other data formats are available (and may be more effeciant for larger datasets).

So let's try reading in our CSV file (with the headers and data)


In [4]:
df_objects = pd.read_csv("pandas.csv")

If the first line doesn't contain the headers, you can tell pandas this:

In [5]:
df_objects = pd.read_csv("pandas.csv", header=None)

otherwise it will "eat" the first line and use them as column names. You can also set the headers yourself:

In [6]:
df_objects = pd.read_csv("pandas.csv", names=['id','Object','Title','Artist','ProductionDateEarliest','Place'])

That should have worked without any output. If Pandas couldn't parse the CSV file it will tell you what caused the parse error.

We've saved the results of reading in the file into df_objects. By default Pandas will create a DataFrame to store the data, which is, much like you would expect, a series of rows of data indexed by either an assigned identifier or a generated incremental identifier (0, 1, 2, 3, etc). If the CSV only contained one index column and one data column this may be overkill, so could instruct Pandas to create a Series data structure instead:

curl -s http://www.vam.ac.uk/api/json/museumobject/search?q=pandas | jq -r '["id", "artist"], (.records[].fields | [.object_number,.artist]) | @csv' > pandas-artist.csv

    series_artists = pd.read_csv("pandas-artist.csv", squeeze=True)

Later we could turn that back into a DataFrame with:

    df_artists = pd.DataFrame(series_artists)

The difference between the two formats will come into XXX

# Basic Operations

So now, we have our data read in, lets take a look at it. We can find how
many rows there are in the normal Pythonic way:


In [5]:
len(df_objects)

10

or we can get some more detailed Pandas information:

In [6]:
df_objects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
id            10 non-null object
object        10 non-null object
title         6 non-null object
artist        10 non-null object
year_start    0 non-null float64
place         10 non-null object
dtypes: float64(1), object(5)
memory usage: 560.0+ bytes


We can get the dimensions of the array (rows, columns)

# TODO 

Pandas will create an index column if we don't give a name of 'id', or tell it which column to use

In [7]:
df_objects.shape

(10, 6)

Which has as we expect 6 columns - id, Object, Title, Artist, ProductionDataEarliest, Place) and 10 data rows 

So, now we know the data has been read in successfully. Let's take a look at it.

The head() & tail() function work much like you would expect (expecially if you are used to the Unix equivalents). They return (by default 5) rows


In [8]:
df_objects.head()

,id,object,title,artist,year_start,place
0,O26744,Card game,pandas' party,Unknown,NaN,England
1,O49687,Set of soft toy pandas,NaN,Fiat,NaN,Asia
2,O1160896,Sculpture,Bruised,Ji Ji,NaN,Shanghai
3,O1177173,Sculpture,Angry,Ji Ji,NaN,Shanghai
4,O1363217,Teddy bear,Panda,unknown,NaN,United Kingdom


In [9]:
df_objects.tail(3)

,id,object,title,artist,year_start,place
7,O1193257,birthday cards,NaN,Hallmark Cards,NaN,Great Britain
8,O1152697,Teddy bear,NaN,Schuco,NaN,Germany
9,O107238,Print,Slottizoo Panda,"Richards, Clifford",NaN,United Kingdom


Or we can show a slice of the data, by row:

In [10]:
df_objects[2:4]

,id,object,title,artist,year_start,place
2,O1160896,Sculpture,Bruised,Ji Ji,NaN,Shanghai
3,O1177173,Sculpture,Angry,Ji Ji,NaN,Shanghai


or by column:

In [11]:
df_objects['title']

0      pandas' party
1                NaN
2            Bruised
3              Angry
4              Panda
5           Jingjing
6                NaN
7                NaN
8                NaN
9    Slottizoo Panda
Name: title, dtype: object

or both:

In [12]:
df_objects['title'][3:5]

3    Angry
4    Panda
Name: title, dtype: object

## Cleaning up the Data


Before we can start doing anything meaningful with the data, we need to get
it into shape. First we should remove any rows that don't contain any meaningful values (i.e. blank fields):

In [13]:
df_objects_clean = df_objects.dropna(inplace=False)
len(df_objects_clean)

0

This has thrown out rows that have blank values (the definition of what is considered a blank value is configurable, see the na_values argument in the [docs](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html). Unfortunaley for us, all the rows have some blank values, so we are left with no data.

A more useful data cleaning option is to tell Pandas which columns we really care about being blank, and to keep the rest. Let's say we always insist on having a title:

In [16]:
df_objects_clean = df_objects.dropna(subset=['title'])
len(df_objects_clean)

6

Now we have all the rows that have an title set, even if other columns are blank.

Finally, if we don't need to keep the original DataFrame (df_objects) we can avoid having to create another variable by passing the inplace=True argument to the function (this works for most functions) and the data clean will operate on the original variable.

Also as part of a data cleanup, we might want to remove any duplicate values. Let's say we only want an artist to appear once in the list. First we need to select the column we will check for duplicates on. We do this by creating a mask array of boolean values

In [23]:
artists = df_objects['Artist']
artists.head()

0                artist
1    Richards, Clifford
2                 Ji Ji
3                 Ji Ji
4                 Ji Ji
Name: Artist, dtype: object

Now we ask Pandas to look for duplicate values in the column, this creates a new column of the same length but now of boolean values, 'True' for a duplicate and 'False' for not. We also pass the option keep='first' to tell Pandas to mark the first occurance of a duplicated value as 'False' (alternatively we could have passed keep='last')

In [20]:
non_duplicates_mask = artists.duplicated(keep='first')
non_duplicates_mask.head()

0    False
1    False
2    False
3     True
4     True
Name: Artist, dtype: bool

Next we negate these values, so we now have a list of duplicate values

In [21]:
duplicates_mask = ~non_duplicates_mask
duplicates_mask.head()

0     True
1     True
2     True
3    False
4    False
Name: Artist, dtype: bool

Finally, we use this new column as a mask, to select from the dataframe those rows to keep and the duplicate rows to remove

In [24]:
df_objects_nodup = df_objects[duplicates_mask]
df_objects_nodup.head()

,id,Object,Title,Artist,ProductionDateEarliest,Place
0,id,object,title,artist,year_start,place
1,O107238,Print,Slottizoo Panda,"Richards, Clifford",1962,United Kingdom
2,O1160896,Sculpture,"Bruised; Hi, Panda",Ji Ji,2007,Shanghai
5,O1261213,Toy panda,NaN,Steiff,1995,Germany
6,O242160,Poster,Wildscreen 90,"Eckersley, Tom",1990,England


Now we only have one object from 'Ji Ji'.

As usual, we can compress all of the work down into a one liner:

In [27]:
df_objects_oneliner = df_objects[~df_objects['Artist'].duplicated(keep='first')]
len(df_objects_oneliner)

11

(If you want to remove duplicates based on the index column, the following works)

In [28]:
df_index_nodup = df_objects[~df_objects.index.duplicated(keep='first')]
len(df_index_nodup)

14

## Dealing with Dates

Panda was created to deal with financial transactions, so the 64 bit datetime
time can go to the microsecond from 1668 to ????. Whilst this may mean Pepy's
diary can be covered in close detail, it's problematic to convert pre-1668 date strings to datetimes. The answer is to instead use the
Period object, which can store the date and time values as year, month, day,
hour minute, second (and microsecond). However, this does mean a comparative
slowdown compared to using machine native datatypes.

XXX Example Period usage

XXX Something about PeriodIndex ?

TODO

## Filtering out unwanted values (isin)

We can filter on columns, for example if we only want to see print objects:

```
drop_cats = ['internal_category', 'another_one']

df = df[~df.category.isin(drop_cats)]
```


In [30]:
prints = df_objects[df_objects['Object'] == "Poster"]
prints.head()

,id,Object,Title,Artist,ProductionDateEarliest,Place
6,O242160,Poster,Wildscreen 90,"Eckersley, Tom",1990,England
7,O1157960,Poster,The Panda Man Film Party: On the occasion of t...,Unknown,2004,Shanghai
14,O109494,Poster,Uh-oh! Pandaman,"Bandi, Zhao",2004,London


We can filter on a list of values:

In [32]:
object_types = ['Poster', 'Print']
prints_and_posters = df_objects[df_objects.Object.isin(object_types)]
prints_and_posters.head()

,id,Object,Title,Artist,ProductionDateEarliest,Place
1,O107238,Print,Slottizoo Panda,"Richards, Clifford",1962,United Kingdom
6,O242160,Poster,Wildscreen 90,"Eckersley, Tom",1990,England
7,O1157960,Poster,The Panda Man Film Party: On the occasion of t...,Unknown,2004,Shanghai
14,O109494,Poster,Uh-oh! Pandaman,"Bandi, Zhao",2004,London


Alternatively, we can keep everything that doesn't match:

In [34]:
object_types = ['Poster', 'Print']
non_prints_and_posters = df_objects[~df_objects.Object.isin(object_types)]
non_prints_and_posters.head()

,id,Object,Title,Artist,ProductionDateEarliest,Place
0,id,object,title,artist,year_start,place
2,O1160896,Sculpture,"Bruised; Hi, Panda",Ji Ji,2007,Shanghai
3,O1177172,Sculpture,"Smiley; Hi, Panda",Ji Ji,2007,Shanghai
4,O1177173,Sculpture,"Angry; Hi, Panda",Ji Ji,2007,Shanghai
5,O1261213,Toy panda,NaN,Steiff,1995,Germany


## Renaming columns

Columns in a dataframe can be easily renamed:

df.columns = ['ColA', 'ColB', 'ColC']


In [35]:
df_objects.columns = ['VAMID', 'a', 'b', 'c', 'd', 'e']
df_objects.head()

,VAMID,a,b,c,d,e
0,id,object,title,artist,year_start,place
1,O107238,Print,Slottizoo Panda,"Richards, Clifford",1962,United Kingdom
2,O1160896,Sculpture,"Bruised; Hi, Panda",Ji Ji,2007,Shanghai
3,O1177172,Sculpture,"Smiley; Hi, Panda",Ji Ji,2007,Shanghai
4,O1177173,Sculpture,"Angry; Hi, Panda",Ji Ji,2007,Shanghai


# Worked Example: Joining two collection datasets

At this point, we have enough Panda Fu (sorry) to start being able to do some
interesting data processing. Let's use two museum collections data sets
made available via CSV and JSON on GitHub (to avoid having to bulk download them
via an API). The data sets contain overlapping details for artists in
both collections, with similiar (but not identical!) columns for artist names,
birth/death etc. Let's try to join the datasets on the artist name, and pull
put two relevant columns from each dataset.

(TODO curl to get from Git)

The Tate dataset has the following fields:

    id,name,gender,dates,yearOfBirth,yearOfDeath,placeOfBirth,placeOfDeath,url

and MoMa has:

    ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN
    
    


In [36]:
    df_tate = pd.read_csv("artist_data.csv")
    df_tate.head(3)

,﻿id,name,gender,dates,yearOfBirth,yearOfDeath,placeOfBirth,placeOfDeath,url
0,10093,"Abakanowicz, Magdalena",Female,born 1930,1930.0,NaN,Polska,NaN,http://www.tate.org.uk/art/artists/magdalena-a...
1,0,"Abbey, Edwin Austin",Male,1852–1911,1852.0,1911.0,"Philadelphia, United States","London, United Kingdom",http://www.tate.org.uk/art/artists/edwin-austi...
2,2756,"Abbott, Berenice",Female,1898–1991,1898.0,1991.0,"Springfield, United States","Monson, United States",http://www.tate.org.uk/art/artists/berenice-ab...


We can immediately see a problem with the yearofBirth/yearOfDeath fields, they've been parsed
as a floating number rather than an integer as we would expect. This is
because Pandas cannot represent Not-a-Number (i.e. an unknown year) as an
integer (see [here](http://pandas.pydata.org/pandas-docs/stable/gotchas.html#nan-integer-na-values-and-na-type-promotions)) so instead it converts it to an integer.

We can fix this by replacing the NaN float64 values with 0.0 and then converting
the column to integers

In [37]:
df_tate['yearOfBirth'].fillna(0, inplace=True)
df_tate['yearOfBirth'] = df_tate['yearOfBirth'].astype(int)

df_tate['yearOfDeath'].fillna(0, inplace=True)
df_tate['yearOfDeath'] = df_tate['yearOfDeath'].astype(int)

df_tate.head(3)


,﻿id,name,gender,dates,yearOfBirth,yearOfDeath,placeOfBirth,placeOfDeath,url
0,10093,"Abakanowicz, Magdalena",Female,born 1930,1930,0,Polska,NaN,http://www.tate.org.uk/art/artists/magdalena-a...
1,0,"Abbey, Edwin Austin",Male,1852–1911,1852,1911,"Philadelphia, United States","London, United Kingdom",http://www.tate.org.uk/art/artists/edwin-austi...
2,2756,"Abbott, Berenice",Female,1898–1991,1898,1991,"Springfield, United States","Monson, United States",http://www.tate.org.uk/art/artists/berenice-ab...


Next we read in the MoMa data:

And looking at the them both:

    df_tate.head()

    df_moma.head()

we can see we have a problem with joining on artist names; in the Tate data
names are given in the form "Surname, Firstname", whereas MoMa uses "Firstname
Surname". Let's do a simple reverse (which will not work for all cases
like "Barret, George, Junior", we'll ignore that for now) and apply that
to the name column in the Tate dataset to create a new column from it


In [38]:
df_moma = pd.read_csv("Artists.csv")
df_moma.head(3)

,﻿ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN
0,1,Robert Arneson,"American, 1930–1992",American,Male,1930,1992,NaN,NaN
1,2,Doroteo Arnaiz,"Spanish, born 1936",Spanish,Male,1936,0,NaN,NaN
2,3,Bill Arnold,"American, born 1941",American,Male,1941,0,NaN,NaN


We can see we have a problem with joining on artist names; in the Tate data
names are given in the form "Surname, Firstname", whereas MoMa uses "Firstname
Surname". Let's do a simple reverse (which will not work for all cases
like "Barret, George, Junior", we'll ignore that for now) and apply that
to the name column in the Tate dataset to create a new column from it

In [39]:
df_tate['name2'] = df_tate['name'].apply(lambda x: " ".join(x.split(", ")[::-1]))
df_tate.head(3)

,﻿id,name,gender,dates,yearOfBirth,yearOfDeath,placeOfBirth,placeOfDeath,url,name2
0,10093,"Abakanowicz, Magdalena",Female,born 1930,1930,0,Polska,NaN,http://www.tate.org.uk/art/artists/magdalena-a...,Magdalena Abakanowicz
1,0,"Abbey, Edwin Austin",Male,1852–1911,1852,1911,"Philadelphia, United States","London, United Kingdom",http://www.tate.org.uk/art/artists/edwin-austi...,Edwin Austin Abbey
2,2756,"Abbott, Berenice",Female,1898–1991,1898,1991,"Springfield, United States","Monson, United States",http://www.tate.org.uk/art/artists/berenice-ab...,Berenice Abbott


which mostly works, except for some values that should have been ignored e.g.

    "Young-Hae Chang Heavy Industries (Young-Hae Chang, Marc Voge)"

will become:

    Marc Voge) Young-Hae Chang Heavy Industries (Young-Hae Chang


But for the moment we will ignore this.

We now have two datasets with an artist name column that should, mostly serve
as a join key. However, there are still variations in spelling/spacing/etc
that will reduce the name of matches if we just do a simple string equality
test. So instead we'll use python's built in (via difflib) get_close_matches
function, which given a string and an list of strings, returns the closest
matches:

In [40]:
import difflib
    
def close_match(name):
 matches = difflib.get_close_matches(name, df_moma['DisplayName'], cutoff=0.9)
 if len(matches) > 0:
  return matches[0]
 else:
  return "N/A"
    
df_tate['name2'] = df_tate['name2'].apply(lambda x: close_match(x))

df_tate.head()

,﻿id,name,gender,dates,yearOfBirth,yearOfDeath,placeOfBirth,placeOfDeath,url,name2
0,10093,"Abakanowicz, Magdalena",Female,born 1930,1930,0,Polska,NaN,http://www.tate.org.uk/art/artists/magdalena-a...,Magdalena Abakanowicz
1,0,"Abbey, Edwin Austin",Male,1852–1911,1852,1911,"Philadelphia, United States","London, United Kingdom",http://www.tate.org.uk/art/artists/edwin-austi...,N/A
2,2756,"Abbott, Berenice",Female,1898–1991,1898,1991,"Springfield, United States","Monson, United States",http://www.tate.org.uk/art/artists/berenice-ab...,Berenice Abbott
3,1,"Abbott, Lemuel Francis",Male,1760–1803,1760,1803,"Leicestershire, United Kingdom","London, United Kingdom",http://www.tate.org.uk/art/artists/lemuel-fran...,N/A
4,622,"Abrahams, Ivor",Male,born 1935,1935,0,"Wigan, United Kingdom",NaN,http://www.tate.org.uk/art/artists/ivor-abraha...,Ivor Abrahams


In [41]:
len(df_tate)

3532

The cutoff value is how far away from the original string matches will be
founf (based on ??). Lowering this will find more matches but potenitally
incorrect. A superior algorithm would be based on understanding of name
stemming)

(this is very slow as it keeps rescanning the list for every row. Suggestions
for improvement welcome.)

and then remove anything that didn't return any matches:

    df_tate.dropna(subset=['name2'], inplace=True)


In [42]:
df_tate.dropna(subset=['name2'], inplace=True)
len(df_tate)
df_tate.head(5)

,﻿id,name,gender,dates,yearOfBirth,yearOfDeath,placeOfBirth,placeOfDeath,url,name2
0,10093,"Abakanowicz, Magdalena",Female,born 1930,1930,0,Polska,NaN,http://www.tate.org.uk/art/artists/magdalena-a...,Magdalena Abakanowicz
1,0,"Abbey, Edwin Austin",Male,1852–1911,1852,1911,"Philadelphia, United States","London, United Kingdom",http://www.tate.org.uk/art/artists/edwin-austi...,N/A
2,2756,"Abbott, Berenice",Female,1898–1991,1898,1991,"Springfield, United States","Monson, United States",http://www.tate.org.uk/art/artists/berenice-ab...,Berenice Abbott
3,1,"Abbott, Lemuel Francis",Male,1760–1803,1760,1803,"Leicestershire, United Kingdom","London, United Kingdom",http://www.tate.org.uk/art/artists/lemuel-fran...,N/A
4,622,"Abrahams, Ivor",Male,born 1935,1935,0,"Wigan, United Kingdom",NaN,http://www.tate.org.uk/art/artists/ivor-abraha...,Ivor Abrahams


TODO fix above not removing N/a

We now have in the tate dataset a subset with only the artists that occur
in both data sets (or rather, have a matching name in both datasets, 'John
Smith' in MoMa may be a different 'John Smith' in the Tate). Let's try merging:

Mext we merge the two datatables. Merging is a huge topic (see here), but
essentialy we have to tell Pandas, what we want to merge, which columns to
merge on, and how to handle gaps from either or both sides).

This will merge on the field 'name2' from df_tate and 'DisplayName' from df_moma, only keeping rows that exist in the tate index [???]




In [43]:
df_museums = pd.merge(df_tate, df_moma, left_on='name2', right_on='DisplayName', how='left')
df_museums.head(1)

,﻿id,name,gender,dates,yearOfBirth,yearOfDeath,placeOfBirth,placeOfDeath,url,name2,﻿ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN
0,10093,"Abakanowicz, Magdalena",Female,born 1930,1930,0,Polska,NaN,http://www.tate.org.uk/art/artists/magdalena-a...,Magdalena Abakanowicz,38.0,Magdalena Abakanowicz,"Polish, born 1930",Polish,Female,1930.0,0.0,Q158080,500084577.0


In [44]:
df_museums.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3537 entries, 0 to 3536
Data columns (total 19 columns):
﻿id               3537 non-null int64
name              3537 non-null object
gender            3421 non-null object
dates             3475 non-null object
yearOfBirth       3537 non-null int64
yearOfDeath       3537 non-null int64
placeOfBirth      3044 non-null object
placeOfDeath      1454 non-null object
url               3537 non-null object
name2             3537 non-null object
﻿ConstituentID    1174 non-null float64
DisplayName       1174 non-null object
ArtistBio         1164 non-null object
Nationality       1156 non-null object
Gender            1158 non-null object
BeginDate         1174 non-null float64
EndDate           1174 non-null float64
Wiki QID          808 non-null object
ULAN              797 non-null float64
dtypes: float64(4), int64(3), object(12)
memory usage: 552.7+ KB


This creates a rather unweildy merge dataset with all the columns from both
datasets, joined on artists.

As this is a bit too much, let's instead just pull out one column from each
dataset for the merge:


In [45]:
df_museums2 = df_museums[["name2", "yearOfBirth", "BeginDate"]]
df_museums2.head(1)

,name2,yearOfBirth,BeginDate
0,Magdalena Abakanowicz,1930,1930.0


Likewise we could pull out other data columns. An exercise in data analysis
would be to pull out what should be matching columns (year of artists birth
for example) and compare the values from both datasets, to test for any
differences that need further investigation.

That's it for an introduction to Pandas. Next, we move onto everyone's
favourite part, data visulation, using the Bokeh library, which also gives us another way
of discovering data inconsistencies and interesting data patterns, this time within the same dataset.


Many thanks for Tate & MoMa for releasing these records.

